# Preprocessing datasets

## 1. Import packages

In [1]:
from pysus.utilities.readdbc import read_dbc

import pandas as pd
import numpy as np
import xlrd
import glob
import os
import re

## 2. Global variables

In [28]:
# Paths
PATHS = dict(
    IN = '../datas/raw/mr-ssa',
    OUT = '../datas/preprocessing/mr-ssa'
) 

# Folders
FOLDERS = dict(
    DESC = 'datasus/description',
    AIRQ = 'airquality/cetrel/interval_1hour', # interval_15min ou interval_1hour
    WTHR = 'weather/cetrel',
    DSUS = 'datasus'
)

## 3. Functions

In [3]:
def show_enum(arr: [str]) -> {int: str}:
    return {idx: value for (idx, value) in enumerate(arr)}

In [4]:
def _filter(df: pd.DataFrame, cols: [str], values: [str]) -> pd.DataFrame:
    f = [False]
    for cc in cols:
        try:
            aux = df[cc].isin(values)
            f |= aux
        except KeyError:
            pass
    
    return df if len(f) == 1 else df[f]

In [5]:
def drop_cols(df: pd.DataFrame, cols: [str]):
    for c in cols:
        try:
            df.drop(c, inplace=True, axis=1)
        except KeyError:
            pass

In [6]:
def drop_empty(row: [str]) -> [str]:
    return [c for c in row if c != '']

## 4. Air quality - preprocessing

### 4.1 [CETREL] Get air quality excel files

In [43]:
path = f"{PATHS['IN']}/{FOLDERS['AIRQ']}/*.xlsx"
f_airq = glob.glob(path)
show_enum(f_airq)

{0: '../datas/raw/mr-ssa/airquality/cetrel/interval_1hour/Monitoramento Salvador - 2011.xlsx',
 1: '../datas/raw/mr-ssa/airquality/cetrel/interval_1hour/Monitoramento Salvador - 2012.xlsx',
 2: '../datas/raw/mr-ssa/airquality/cetrel/interval_1hour/Monitoramento Salvador - 2013.xlsx',
 3: '../datas/raw/mr-ssa/airquality/cetrel/interval_1hour/Monitoramento Salvador - 2014.xlsx',
 4: '../datas/raw/mr-ssa/airquality/cetrel/interval_1hour/Monitoramento Salvador - 2015.xlsx',
 5: '../datas/raw/mr-ssa/airquality/cetrel/interval_1hour/Monitoramento Salvador - NO.xlsx'}

### 4.2 [CETREL] Generating csv

#### [15min]

In [4]:
for file in f_airq:
    print(f'\nStarting {file}')

    # Open excel file
    wb = xlrd.open_workbook(file)

    # Get station name
    station = file.split('/')[-1]
    station = station[:-5]

    # Get all sheets
    sheet_names = wb.sheet_names()

    for sheet_name in sheet_names:
        # Get sheet
        sheet = wb.sheet_by_name(sheet_name)

        # Rows number
        nrows = sheet.nrows

        # Get cols names
        cols = sheet.row_values(2)

        # Get units
        units = sheet.row_values(3)

        # Concatenate cols with units except cols[0] (date & time)
        cols[1:] = [f'{cols[i]} ({units[i]})' for i in range(1, len(cols))]

        # Init dataframe
        df = pd.DataFrame(columns=cols)

        # Get only valid datas
        for r in range(4, nrows-8):
            aux = pd.DataFrame([sheet.row_values(r)], columns=cols)
            df = pd.concat([df, aux], ignore_index=True)

        # Generate output name
        f_out = f"{PATHS['OUT']}/{FOLDERS['AIRQ']}/{station}-{sheet_name}"

        # Generate gzip output file
        df.to_csv(f'{f_out}.csv.gz', index=False, compression='gzip')

        print(f'[DONE] {f_out}.csv.gz')

    print(f'Ending {file}\n')
print('Ending all.')


Starting ../datas/raw/mr-ssa/airquality/av-acm-detran.xlsx
[DONE] ../datas/preprocessing/mr-ssa/airquality/av-acm-detran-2013.zip
[DONE] ../datas/preprocessing/mr-ssa/airquality/av-acm-detran-2014.zip
[DONE] ../datas/preprocessing/mr-ssa/airquality/av-acm-detran-2015.zip
Ending ../datas/raw/mr-ssa/airquality/av-acm-detran.xlsx


Starting ../datas/raw/mr-ssa/airquality/barros-reis.xlsx
[DONE] ../datas/preprocessing/mr-ssa/airquality/barros-reis-2013.zip
[DONE] ../datas/preprocessing/mr-ssa/airquality/barros-reis-2014.zip
[DONE] ../datas/preprocessing/mr-ssa/airquality/barros-reis-2015.zip
Ending ../datas/raw/mr-ssa/airquality/barros-reis.xlsx


Starting ../datas/raw/mr-ssa/airquality/campo-grande.xlsx
[DONE] ../datas/preprocessing/mr-ssa/airquality/campo-grande-2013.zip
[DONE] ../datas/preprocessing/mr-ssa/airquality/campo-grande-2014.zip
[DONE] ../datas/preprocessing/mr-ssa/airquality/campo-grande-2015.zip
Ending ../datas/raw/mr-ssa/airquality/campo-grande.xlsx


Starting ../datas/raw

#### [1hour]

In [44]:
for file in f_airq:
    print(f'\nStarting {file}')

    # Open excel file
    wb = xlrd.open_workbook(file)

    # Get complement in file name
    complement = file.split('- ')[-1]
    complement = complement[:-5]

    # Get all sheets (stations)
    sheet_names = wb.sheet_names()

    for station in sheet_names:
        # Get sheet (station)
        sheet = wb.sheet_by_name(station)

        # Rows number
        nrows = sheet.nrows

        # Get cols names
        cols = sheet.row_values(10)
        cols[0] = "Date & Time"
        cols = drop_empty(cols)

        # Get units
        units = sheet.row_values(11)

        # Concatenate cols with units except cols[0] (date & time)
        cols[1:] = [f'{cols[i]} ({units[i]})' for i in range(1, len(cols))]

        # Init dataframe
        df = pd.DataFrame(columns=cols)

        # Get only valid datas
        for r in range(12, nrows-8):
            aux = pd.DataFrame([drop_empty(sheet.row_values(r))], columns=cols)
            df = pd.concat([df, aux], ignore_index=True)
            
        # Generate output name
        f_out = f"{PATHS['OUT']}/{FOLDERS['AIRQ']}/{station}-{complement}"

        # Generate gzip output file
        df.to_csv(f'{f_out}.csv.gz', index=False, compression='gzip')

        print(f'[DONE] {f_out}.csv.gz')

    print(f'Ending {file}\n')
print('Ending all.')


Starting ../datas/raw/mr-ssa/airquality/cetrel/interval_1hour/Monitoramento Salvador - 2011.xlsx
[DONE] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/CAMPO GRANDE-2011.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/DIQUE DO TORORÓ-2011.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/PARALELA-CAB-2011.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/PIRAJÁ-2011.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/RIO VERMELHO-2011.csv.gz
Ending ../datas/raw/mr-ssa/airquality/cetrel/interval_1hour/Monitoramento Salvador - 2011.xlsx


Starting ../datas/raw/mr-ssa/airquality/cetrel/interval_1hour/Monitoramento Salvador - 2012.xlsx
[DONE] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/CAMPO GRANDE-2012.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/DIQUE DO TORORÓ-2012.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/airqua

#### [1hour] merging

In [45]:
path = f"{PATHS['OUT']}/{FOLDERS['AIRQ']}/*-20*.gz"
stations = np.unique([f[:-12].split('/')[-1] for f in glob.glob(path)])
stations

array(['AV ACM - DETRAN', 'AV BARROS REIS', 'CAMPO GRANDE',
       'DIQUE DO TORORÓ', 'ITAIGARA', 'PARALELA-CAB', 'PIRAJÁ',
       'RIO VERMELHO'], dtype='<U15')

In [46]:
for station in stations:
    print(f'\nStarting {station}')
    
    path = f"{PATHS['OUT']}/{FOLDERS['AIRQ']}/{station}-20*.gz"
    f_airq = glob.glob(path)
    
    no = f"{PATHS['OUT']}/{FOLDERS['AIRQ']}/{station}-NO*.gz"
    f_no = glob.glob(no)
    
    print('[INFO] Merging years')
    df = pd.DataFrame()
    for file in f_airq:
        aux = pd.read_csv(file, index_col='Date & Time', compression='gzip')
        df = pd.concat([df, aux])
        os.remove(file)
        print(f'[INFO] {file} removed')

    print('[INFO] Merging NO specie')
    for file in f_no:
        aux = pd.read_csv(file, index_col='Date & Time', compression='gzip')
        df = df.join(aux, on='Date & Time')
        os.remove(file)
        print(f'[INFO] {file} removed')
            
    # Generate output name
    f_out = f"{PATHS['OUT']}/{FOLDERS['AIRQ']}/{station}"
            
    # Generate gzip output file
    df.to_csv(f'{f_out}.csv.gz', compression='gzip')

    print(f'[DONE] {f_out}.csv.gz')

    print(f'Ending {station}\n')
print('Ending all.')


Starting AV ACM - DETRAN
[INFO] Merging years
[INFO] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/AV ACM - DETRAN-2013.csv.gz removed
[INFO] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/AV ACM - DETRAN-2014.csv.gz removed
[INFO] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/AV ACM - DETRAN-2015.csv.gz removed
[INFO] Merging NO specie
[INFO] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/AV ACM - DETRAN-NO.csv.gz removed
[DONE] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/AV ACM - DETRAN.csv.gz
Ending AV ACM - DETRAN


Starting AV BARROS REIS
[INFO] Merging years
[INFO] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/AV BARROS REIS-2013.csv.gz removed
[INFO] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/AV BARROS REIS-2014.csv.gz removed
[INFO] ../datas/preprocessing/mr-ssa/airquality/cetrel/interval_1hour/AV BARROS REIS-2015.csv.gz removed
[INFO] Merging NO specie
[IN

## 5. Weather - preprocessing

### 5.1 [CETREL] Get weather excel files

In [30]:
path = f"{PATHS['IN']}/{FOLDERS['WTHR']}/*.xls"
f_wthr = glob.glob(path)
show_enum(f_wthr)

{0: '../datas/raw/mr-ssa/weather/cetrel/rmar-2013.xls',
 1: '../datas/raw/mr-ssa/weather/cetrel/rmar-2014.xls',
 2: '../datas/raw/mr-ssa/weather/cetrel/rmar-2015.xls'}

### 5.2 [CETREL] Generating csv

In [6]:
for file in f_wthr:
    print(f'\nStarting {file}')

    # Open excel file
    wb = xlrd.open_workbook(file)

    # Get year
    year = file.split('/')[-1]
    year = year[-8:-4]

    # Get sheet
    sheet = wb.sheet_by_index(0)

    # Rows number
    nrows = sheet.nrows

    # Get cols names with counts
    stations = sheet.row_values(2, start_colx=1)
    stations_u = list(dict.fromkeys(stations))
    stations_c = { x: stations.count(x) for x in stations_u}

    index_col = sheet.cell_value(3,0)

    colx = dict(start_colx=1, end_colx=0)
    for st in stations_c:
        colx['end_colx'] = colx['start_colx'] + stations_c[st]

        # Get cols names
        cols = [index_col]
        cols.extend(sheet.row_values(3, **colx))

        # Get units
        units = sheet.row_values(4, **colx)

        # Concatenate cols with units
        for i in range(1, len(cols)):
            cols[i] = f'{cols[i]} ({units[i-1]})'

        # Init dataframe
        df = pd.DataFrame(columns=cols)

        # Get only valid datas
        for r in range(5, nrows-8):
            register = [sheet.cell_value(r, 0)]
            register.extend(sheet.row_values(r, **colx))
            aux = pd.DataFrame([register], columns=cols)
            df = pd.concat([df, aux], ignore_index=True)

        # Generate output name
        f_out = f"{PATHS['OUT']}/{FOLDERS['WTHR']}/{st}-{year}.csv.gz"

        # Generate gzip output file
        df.to_csv(f_out, index=False, compression='gzip')

        colx['start_colx'] = colx['end_colx']
        print(f'[DONE] {f_out}')

    print(f'Ending {file}\n')
print('Ending all.') 


Starting ../datas/raw/mr-ssa/weather/rmar-2013.xls
[DONE] ../datas/preprocessing/mr-ssa/weather/AV. ACM - DETRAN-2013.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/AV. BARROS REIS-2013.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/CAMPO GRANDE-2013.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/DIQUE DO TORORO-2013.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/ITAIGARA-2013.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/PARALELA-CAB-2013.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/PIRAJA-2013.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/RIO VERMELHO-2013.csv.gz
Ending ../datas/raw/mr-ssa/weather/rmar-2013.xls


Starting ../datas/raw/mr-ssa/weather/rmar-2014.xls
[DONE] ../datas/preprocessing/mr-ssa/weather/AV. ACM - DETRAN-2014.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/AV. BARROS REIS-2014.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/CAMPO GRANDE-2014.csv.gz
[DONE] ../datas/preprocessing/mr-ssa/weather/DIQUE DO TORORO-2014.csv

## 6. Datasus - preprocessing

### 6.1 Descriptions files
open cnv files in https://filext.com/file-extension/CNV

### 6.2 AIH Reduzida (RD)

SEQ|NOME DO CAMPO|TIPO E TAM|Descrição/Observações
---|-------------|----------|---------------------
<span style="color:red">1|<span style="color:red">UF_ZI|<span style="color:red">char(6)|<span style="color:red">Município Gestor
<span style="color:red">2|<span style="color:red">ANO_CMPT|<span style="color:red">char(4)|<span style="color:red">Ano de processamento da AIH, no formato aaaa.
<span style="color:red">3|<span style="color:red">MÊS_CMPT|<span style="color:red">char(2)|<span style="color:red">Mês de processamento da AIH, no formato mm.
<span style="color:red">4|<span style="color:red">ESPEC|<span style="color:red">char(2)|<span style="color:red">Especialidade do Leito
5|CGC_HOSP|char(14)|CNPJ do Estabelecimento.
6|N_AIH|char(13)|Número da AIH.
<span style="color:red">7|<span style="color:red">IDENT|<span style="color:red">char(1)|<span style="color:red">Identificação do tipo da AIH.
8|CEP|char(8)|CEP do paciente.
9|MUNIC_RES|char(6)|Município de Residência do Paciente
10|NASC|char(8)|Data de nascimento do paciente (aaaammdd).
11|SEXO|char(1)|Sexo do paciente.
<span style="color:red">12|<span style="color:red">UTI_MES_IN|<span style="color:red">numeric(2)|<span style="color:red">Zerado
<span style="color:red">13|<span style="color:red">UTI_MES_AN|<span style="color:red">numeric(2)|<span style="color:red">Zerado
<span style="color:red">14|<span style="color:red">UTI_MES_AL|<span style="color:red">numeric(2)|<span style="color:red">Zerado
15|UTI_MES_TO|numeric(3)|Quantidade de dias de UTI no mês.
<span style="color:red">16|<span style="color:red">MARCA_UTI|<span style="color:red">char(2)|<span style="color:red">Indica qual o tipo de UTI utilizada pelo paciente.
<span style="color:red">17|<span style="color:red">UTI_INT_IN|<span style="color:red">numeric(2)|<span style="color:red">Zerado
<span style="color:red">18|<span style="color:red">UTI_INT_AN|<span style="color:red">numeric(2)|<span style="color:red">Zerado
<span style="color:red">19|<span style="color:red">UTI_INT_AL|<span style="color:red">numeric(2)|<span style="color:red">Zerado
20|UTI_INT_TO|numeric(3)|Quantidade de diárias em unidade intermediaria.
<span style="color:red">21|<span style="color:red">DIAR_ACOM|<span style="color:red">numeric(3)|<span style="color:red">Quantidade de diárias de acompanhante.
<span style="color:red">22|<span style="color:red">QT_DIARIAS|<span style="color:red">numeric(3)|<span style="color:red">Quantidade de diárias.
<span style="color:red">23|<span style="color:red">PROC_SOLIC|<span style="color:red">char(10)|<span style="color:red">Procedimento solicitado.
<span style="color:red">24|<span style="color:red">PROC_REA|<span style="color:red">char(10)|<span style="color:red">Procedimento realizado.
25|VAL_SH|numeric(13,2)|Valor de serviços hospitalares.
26|VAL_SP|numeric(13,2)|Valor de serviços profissionais.
<span style="color:red">27|<span style="color:red">VAL_SADT|<span style="color:red">numeric(13,2)|<span style="color:red">Zerado
<span style="color:red">28|<span style="color:red">VAL_RN|<span style="color:red">numeric(13,2)|<span style="color:red">Zerado
<span style="color:red">29|<span style="color:red">VAL_ACOMP|<span style="color:red">numeric(13,2)|<span style="color:red">Zerado
<span style="color:red">30|<span style="color:red">VAL_ORTP|<span style="color:red">numeric(13,2)|<span style="color:red">Zerado
<span style="color:red">31|<span style="color:red">VAL_SANGUE|<span style="color:red">numeric(13,2)|<span style="color:red">Zerado
<span style="color:red">32|<span style="color:red">VAL_SADTSR|<span style="color:red">numeric(11,2)|<span style="color:red">Zerado
<span style="color:red">33|<span style="color:red">VAL_TRANSP|<span style="color:red">numeric(13,2)|<span style="color:red">Zerado
<span style="color:red">34|<span style="color:red">VAL_OBSANG|<span style="color:red">numeric(11,2)|<span style="color:red">Zerado
<span style="color:red">35|<span style="color:red">VAL_PED1AC|<span style="color:red">numeric(11,2)|<span style="color:red">Zerado
<span style="color:red">36|<span style="color:red">VAL_TOT|<span style="color:red">numeric(14,2)|<span style="color:red">Valor total da AIH.
<span style="color:red">37|<span style="color:red">VAL_UTI|<span style="color:red">numeric(8,2)|<span style="color:red">Valor de UTI.
<span style="color:red">38|<span style="color:red">US_TOT|<span style="color:red">numeric(10,2)|<span style="color:red">Valor total, em dólar.
39|DT_INTER|char(8)|Data de internação no formato aaammdd.
<span style="color:red">40|<span style="color:red">DT_SAIDA|<span style="color:red">char(8)|<span style="color:red">Data de saída, no formato aaaammdd.
41|DIAG_PRINC|char(4)|Código do diagnóstico principal (CID10).
42|DIAG_SECUN|char(4)|Código do diagnostico secundário (CID10). Preenchido com zeros a partir de 201501.
<span style="color:red">43|<span style="color:red">COBRANCA|<span style="color:red">char(2)|<span style="color:red">Motivo de Saída/Permanência
<span style="color:red">44|<span style="color:red">NATUREZA|<span style="color:red">char(2)|<span style="color:red">Natureza jurídica do hospital (com conteúdo até maio/12). Era utilizada a classificação de Regime e Natureza.
<span style="color:red">45|<span style="color:red">NAT_JUR|<span style="color:red">char(4)|<span style="color:red">Natureza jurídica do Estabelecimento, conforme a Comissão Nacional de Classificação - CONCLA
<span style="color:red">46|<span style="color:red">GESTAO|<span style="color:red">char(1)|<span style="color:red">Indica o tipo de gestão do hospital.
<span style="color:red">47|<span style="color:red">RUBRICA|<span style="color:red">numeric(5)|<span style="color:red">Zerado
<span style="color:red">48|<span style="color:red">IND_VDRL|<span style="color:red">char(1)|<span style="color:red">Indica exame VDRL.
49|MUNIC_MOV|char(6)|Município do Estabelecimento.
<span style="color:red">50|<span style="color:red">COD_IDADE|<span style="color:red">char(1)|<span style="color:red">Unidade de medida da idade.
<span style="color:red">51|<span style="color:red">IDADE|<span style="color:red">numeric(2)|<span style="color:red">Idade.
52|DIAS_PERM|numeric(5)|Dias de Permanência.
53|MORTE|numeric(1)|Indica Óbito
54|NACIONAL|char(2)|Código da nacionalidade do paciente.
<span style="color:red">55|<span style="color:red">NUM_PROC|<span style="color:red">char(4)|<span style="color:red">Zerado
<span style="color:red">56|<span style="color:red">CAR_INT|<span style="color:red">char(2)|<span style="color:red">Caráter da internação.
<span style="color:red">57|<span style="color:red">TOT_PT_SP|<span style="color:red">numeric(6)|<span style="color:red">Zerado
<span style="color:red">58|<span style="color:red">CPF_AUT|<span style="color:red">char(11)|<span style="color:red">Zerado
<span style="color:red">59|<span style="color:red">HOMONIMO|<span style="color:red">char(1)|<span style="color:red">Indicador se o paciente da AIH é homônimo do paciente de outra AIH.
<span style="color:red">60|<span style="color:red">NUM_FILHOS|<span style="color:red">numeric(2)|<span style="color:red">Número de filhos do paciente.
61|INSTRU|char(1)|Grau de instrução do paciente.
<span style="color:red">62|<span style="color:red">CID_NOTIF|<span style="color:red">char(4)|<span style="color:red">CID de Notificação.
<span style="color:red">63|<span style="color:red">CONTRACEP1|<span style="color:red">char(2)|<span style="color:red">Tipo de contraceptivo utilizado.
<span style="color:red">64|<span style="color:red">CONTRACEP2|<span style="color:red">char(2)|<span style="color:red">Segundo tipo de contraceptivo utilizado.
<span style="color:red">65|<span style="color:red">GESTRISCO|<span style="color:red">char(1)|<span style="color:red">Indicador se é gestante de risco.
66|INSC_PN|char(12)|Número da gestante no pré-natal.
<span style="color:red">67|<span style="color:red">SEQ_AIH5|<span style="color:red">char(3)|<span style="color:red">Sequencial de longa permanência (AIH tipo 5).
<span style="color:red">68|<span style="color:red">CBOR|<span style="color:red">char(3)|<span style="color:red">Ocupação do paciente, segundo a Classificação Brasileira de Ocupações – CBO.
<span style="color:red">69|<span style="color:red">CNAER|<span style="color:red">char(3)|<span style="color:red">Código de acidente de trabalho.
<span style="color:red">70|<span style="color:red">VINCPREV|<span style="color:red">char(1)|<span style="color:red">Vínculo com a Previdência.
<span style="color:red">71|<span style="color:red">GESTOR_COD|<span style="color:red">char(3)|<span style="color:red">Motivo de autorização da AIH pelo Gestor.
<span style="color:red">72|<span style="color:red">GESTOR_TP|<span style="color:red">char(1)|<span style="color:red">Tipo de gestor.
<span style="color:red">73|<span style="color:red">GESTOR_CPF|<span style="color:red">char(11)|<span style="color:red">Número do CPF do Gestor.
<span style="color:red">74|<span style="color:red">GESTOR_DT|<span style="color:red">char(8)|<span style="color:red">Data da autorização dada pelo Gestor (aaaammdd).
75|CNES|char(7)|Código CNES do hospital.
<span style="color:red">76|<span style="color:red">CNPJ_MANT|<span style="color:red">char(14)|<span style="color:red">CNPJ da mantenedora.
<span style="color:red">77|<span style="color:red">INFEHOSP|<span style="color:red">char(1)|<span style="color:red">Status de infecção hospitalar.
<span style="color:red">78|<span style="color:red">CID_ASSO|<span style="color:red">char(4)|<span style="color:red">CID causa.
<span style="color:red">79|<span style="color:red">CID_MORTE|<span style="color:red">char(4)|<span style="color:red">CID da morte.
<span style="color:red">80|<span style="color:red">COMPLEX|<span style="color:red">char(2)|<span style="color:red">Complexidade.
<span style="color:red">81|<span style="color:red">FINANC|<span style="color:red">char(2)|<span style="color:red">Tipo de financiamento.
<span style="color:red">82|<span style="color:red">FAEC_TP|<span style="color:red">char(6)|<span style="color:red">Subtipo de financiamento FAEC.
<span style="color:red">83|<span style="color:red">REGCT|<span style="color:red">char(4)|<span style="color:red">Regra contratual.
84|RACA_COR|char(4)|Raça/Cor do paciente.
85|ETNIA|char(4)|Etnia do paciente, se raça cor for indígena.
<span style="color:red">86|<span style="color:red">SEQUENCIA|<span style="color:red">numeric(9)|<span style="color:red">Sequencial da AIH na remessa.
<span style="color:red">87|<span style="color:red">REMESSA|<span style="color:red">char(21)|<span style="color:red">Número da remessa.
<span style="color:red">88|<span style="color:red">AUD_JUST|<span style="color:red">char (50)|<span style="color:red">Justificativa do auditor para aceitação da AIH sem o número do Cartão Nacional de Saúde.
<span style="color:red">89|<span style="color:red">SIS_JUST|<span style="color:red">char (50)|<span style="color:red">Justificativa do estabelecimento para aceitação da AIH sem o número do Cartão Nacional de Saúde.
<span style="color:red">90|<span style="color:red">VAL_SH_FED|<span style="color:red">numeric (10, 2)|<span style="color:red">Valor do complemento federal de serviços hospitalares. Está incluído no valor total da AIH.
<span style="color:red">91|<span style="color:red">VAL_SP_FED|<span style="color:red">numeric (10, 2)|<span style="color:red">Valor do complemento federal de serviços profissionais. Está incluído no valor total da AIH.
<span style="color:red">92|<span style="color:red">VAL_SH_GES|<span style="color:red">numeric (10, 2)|<span style="color:red">Valor do complemento do gestor (estadual ou municipal) de serviços hospitalares. Está incluído no valor total da AIH.
<span style="color:red">93|<span style="color:red">VAL_SP_GES|<span style="color:red">numeric (10, 2)|<span style="color:red">Valor do complemento do gestor (estadual ou municipal) de serviços profissionais. Está incluído no valor total da AIH.
<span style="color:red">94|<span style="color:red">VAL_UCI|<span style="color:red">numeric (10, 2)|<span style="color:red">Valor de UCI.
<span style="color:red">95|<span style="color:red">MARCA_UCI|<span style="color:red">char (2)|<span style="color:red">Tipo de UCI utilizada pelo paciente.
96|DIAGSEC1|char (4)|Diagnóstico secundário1.
97|DIAGSEC2|char (4)|Diagnóstico secundário 2.
98|DIAGSEC3|char (4)|Diagnóstico secundário 3.
99|DIAGSEC4|char (4)|Diagnóstico secundário 4.
100|DIAGSEC5|char (4)|Diagnóstico secundário 5.
101|DIAGSEC6|char (4)|Diagnóstico secundário 6.
102|DIAGSEC7|char (4)|Diagnóstico secundário 7.
103|DIAGSEC8|char (4)|Diagnóstico secundário 8.
104|DIAGSEC9|char (4)|Diagnóstico secundário 9.
<span style="color:red">105|<span style="color:red">TPDISEC1|<span style="color:red">char(1)|<span style="color:red">Tipo de diagnóstico secundário 1.
<span style="color:red">107|<span style="color:red">TPDISEC2|<span style="color:red">char(1)|<span style="color:red">Tipo de diagnóstico secundário 2.
<span style="color:red">108|<span style="color:red">TPDISEC3|<span style="color:red">char(1)|<span style="color:red">Tipo de diagnóstico secundário 3.
<span style="color:red">109|<span style="color:red">TPDISEC4|<span style="color:red">char(1)|<span style="color:red">Tipo de diagnóstico secundário 4.
<span style="color:red">110|<span style="color:red">TPDISEC5|<span style="color:red">char(1)|<span style="color:red">Tipo de diagnóstico secundário 5.
<span style="color:red">111|<span style="color:red">TPDISEC6|<span style="color:red">char(1)|<span style="color:red">Tipo de diagnóstico secundário 6.
<span style="color:red">112|<span style="color:red">TPDISEC7|<span style="color:red">char(1)|<span style="color:red">Tipo de diagnóstico secundário 7.
<span style="color:red">113|<span style="color:red">TPDISEC8|<span style="color:red">char(1)|<span style="color:red">Tipo de diagnóstico secundário 8.
<span style="color:red">114|<span style="color:red">TPDISEC9|<span style="color:red">char(1)|<span style="color:red">Tipo de diagnóstico secundário 9.

### 6.3 List of filters

#### 6.3.1 Unused columns

In [31]:
unused_cols = [
    'UF_ZI',
    'ANO_CMPT',
    'MES_CMPT',
    'ESPEC',
    'IDENT',
    'UTI_MES_IN',
    'UTI_MES_AN',
    'UTI_MES_AL',
    'MARCA_UTI',
    'UTI_INT_IN',
    'UTI_INT_AN',
    'UTI_INT_AL',
    'DIAR_ACOM',
    'QT_DIARIAS',
    'PROC_SOLIC',
    'PROC_REA',
    'VAL_SADT',
    'VAL_RN',
    'VAL_ACOMP',
    'VAL_ORTP',
    'VAL_SANGUE',
    'VAL_SADTSR',
    'VAL_TRANSP',
    'VAL_OBSANG',
    'VAL_PED1AC',
    'VAL_TOT',
    'VAL_UTI',
    'US_TOT',
    'DT_SAIDA',
    'COBRANCA',
    'NATUREZA',
    'NAT_JUR',
    'GESTAO',
    'RUBRICA',
    'IND_VDRL',
    'COD_IDADE',
    'IDADE',
    'NUM_PROC',
    'CAR_INT',
    'TOT_PT_SP',
    'CPF_AUT',
    'HOMONIMO',
    'NUM_FILHOS',
    'CID_NOTIF',
    'CONTRACEP1',
    'CONTRACEP2',
    'GESTRISCO',
    'SEQ_AIH5',
    'CBOR',
    'CNAER',
    'VINCPREV',
    'GESTOR_COD',
    'GESTOR_TP',
    'GESTOR_CPF',
    'GESTOR_DT',
    'CNPJ_MANT',
    'INFEHOSP',
    'CID_ASSO',
    'CID_MORTE',
    'COMPLEX',
    'FINANC',
    'FAEC_TP',
    'REGCT',
    'SEQUENCIA',
    'REMESSA',
    'AUD_JUST',
    'SIS_JUST',
    'VAL_SH_FED',
    'VAL_SP_FED',
    'VAL_SH_GES',
    'VAL_SP_GES',
    'VAL_UCI',
    'MARCA_UCI',
    'TPDISEC1',
    'TPDISEC2',
    'TPDISEC3',
    'TPDISEC4',
    'TPDISEC5',
    'TPDISEC6',
    'TPDISEC7',
    'TPDISEC8',
    'TPDISEC9',
]

#### 6.3.2 Cities (Metropolitan Region of Salvador)

In [32]:
cities_mrs = {
    '292740': 'Salvador',
    '290570': 'Camaçari',
    '292920': 'São Francisco do Conde',
    '291920': 'Lauro de Freitas',
    '293070': 'Simões Filho',
    '290650': 'Candeias',
    '291005': "Dias d'Ávila",
    '292100': 'Mata de São João',
    '292520': 'Pojuca',
    '292950': 'São Sebastião do Passé',
    '293320': 'Vera Cruz',
    '291992': 'Madre de Deus',
    '291610': 'Itaparica'
}
cities_cols = ['MUNIC_RES', 'MUNIC_MOV']

#### 6.3.3 Respiratory diseases

In [33]:
resp_diseases = {
    'J00': 'Nasofaringite aguda                          J00 -J009',
    'J01': 'Sinusite aguda                               J01 -J019',
    'J010': 'Sinusite maxilar aguda                       J010',
    'J011': 'Sinusite frontal aguda                       J011',
    'J012': 'Sinusite etmoidal aguda                      J012',
    'J013': 'Sinusite esfenoidal aguda                    J013',
    'J014': 'Pansinusite aguda                            J014',
    'J018': 'Outr sinusites agudas                        J018',
    'J019': 'Sinusite aguda NE                            J019',
    'J02': 'Faringite aguda                              J02 -J029',
    'J020': 'Faringite estreptococica                     J020',
    'J028': 'Faringite aguda dev outr microorg espec      J028',
    'J029': 'Faringite aguda NE                           J029',
    'J03': 'Amigdalite aguda                             J03 -J039',
    'J030': 'Amigdalite estreptococica                    J030',
    'J038': 'Amigdalite aguda dev outr microorg espec     J038',
    'J039': 'Amigdalite aguda NE                          J039',
    'J04': 'Laringite e traqueite agudas                 J04 -J049',
    'J040': 'Laringite aguda                              J040',
    'J041': 'Traqueite aguda                              J041',
    'J042': 'Laringotraqueite aguda                       J042',
    'J05': 'Laringite obstrutiva aguda e epiglotite      J05 -J059',
    'J050': 'Laringite obstrutiva aguda                   J050',
    'J051': 'Epiglotite aguda                             J051',
    'J06': 'Infecc agudas vias aereas super loc mult NE  J06 -J069',
    'J060': 'Laringofaringite aguda                       J060',
    'J068': 'Outr infecc agudas vias aereas sup loc mult  J068',
    'J069': 'Infecc aguda das vias aereas super NE        J069',
    'J09': 'Influenza dev virus gripe aviária            J09 -J099',
    'J10': 'Influenza dev outro virus influenza ident    J10 -J109',
    'J100': 'Influenza c/pneum dev outr virus influ ident J100',
    'J101': 'Influenz c/out manif resp out vir infl ident J101',
    'J108': 'Influenza c/out manif dev out vir infl ident J108',
    'J11': 'Influenza dev virus nao identificado         J11 -J119',
    'J110': 'Influenza c/pneumonia dev virus nao ident    J110',
    'J111': 'Influenza c/out manif resp dev virus n ident J111',
    'J118': 'Influenza c/outr manif dev virus nao ident   J118',
    'J12': 'Pneumonia viral NCOP                         J12 -J129',
    'J120': 'Pneumonia dev adenovirus                     J120',
    'J121': 'Pneumonia dev virus respirat sincicial       J121',
    'J122': 'Pneumonia dev parainfluenza                  J122',
    'J128': 'Outr pneumonias virais                       J128',
    'J129': 'Pneumonia viral NE                           J129',
    'J13': 'Pneumonia dev Streptococcus pneumoniae       J13 -J139',
    'J14': 'Pneumonia dev Haemophilus infuenzae          J14 -J149',
    'J15': 'Pneumonia bacter NCOP                        J15 -J159',
    'J150': 'Pneumonia dev Klebsiella pneumoniae          J150',
    'J151': 'Pneumonia dev Pseudomonas                    J151',
    'J152': 'Pneumonia dev Staphylococcus                 J152',
    'J153': 'Pneumonia dev Streptococcus do grupo B       J153',
    'J154': 'Pneumonia dev outr estreptococos             J154',
    'J155': 'Pneumonia dev Escherichia coli               J155',
    'J156': 'Pneumonia dev outr bact aerobicas gram-neg   J156',
    'J157': 'Pneumonia dev Mycoplasma pneumoniae          J157',
    'J158': 'Outr pneumonias bacter                       J158',
    'J159': 'Pneumonia bacter NE                          J159',
    'J16': 'Pneumonia dev out microorg infecc espec NCOP J16 -J169',
    'J160': 'Pneumonia dev clamidias                      J160',
    'J168': 'Pneumonia dev outr microorg infecc espec     J168',
    'J17': 'Pneumonia em doenc COP                       J17 -J179',
    'J170': 'Pneumonia em doenc bacter COP                J170',
    'J171': 'Pneumonia em doenc virais COP                J171',
    'J172': 'Pneumonia em micoses COP                     J172',
    'J173': 'Pneumonia em doenc parasit COP               J173',
    'J178': 'Pneumonia em outr doenc COP                  J178',
    'J18': 'Pneumonia p/microorg NE                      J18 -J189',
    'J180': 'Broncopneumonia NE                           J180',
    'J181': 'Pneumonia lobar NE                           J181',
    'J182': 'Pneumonia hipostatica NE                     J182',
    'J188': 'Outr pneumonias dev microorg NE              J188',
    'J189': 'Pneumonia NE                                 J189',
    'J20': 'Bronquite aguda                              J20 -J209',
    'J200': 'Bronquite aguda dev Mycoplasma pneumoniae    J200',
    'J201': 'Bronquite aguda dev Haemophilus influenzae   J201',
    'J202': 'Bronquite aguda dev estreptococos            J202',
    'J203': 'Bronquite aguda dev virus Coxsackie          J203',
    'J204': 'Bronquite aguda dev virus parainfluenza      J204',
    'J205': 'Bronquite aguda dev virus sincicial respirat J205',
    'J206': 'Bronquite aguda dev rinovirus                J206',
    'J207': 'Bronquite aguda dev echovirus                J207',
    'J208': 'Bronquite aguda dev outr microorg espec      J208',
    'J209': 'Bronquite aguda NE                           J209',
    'J21': 'Bronquiolite aguda                           J21 -J219',
    'J210': 'Bronquiolite aguda dev virus sincicial resp  J210',
    'J218': 'Bronquiolite aguda dev outr microorg espec   J218',
    'J219': 'Bronquite aguda NE                           J219',
    'J22': 'Infecc agudas NE das vias aereas infer       J22 -J229',
    'J30': 'Rinite alergica e vasomotora                 J30 -J309',
    'J300': 'Rinite vasomotora                            J300',
    'J301': 'Rinite alergica dev polen                    J301',
    'J302': 'Outr rinites alergicas sazonais              J302',
    'J303': 'Outr rinites alergicas                       J303',
    'J304': 'Rinite alergica NE                           J304',
    'J31': 'Rinite nasofaringite e faringite cronicas    J31 -J319',
    'J310': 'Rinite cronica                               J310',
    'J311': 'Nasofaringite cronica                        J311',
    'J312': 'Faringite cronica                            J312',
    'J32': 'Sinusite cronica                             J32 -J329',
    'J320': 'Sinusite maxilar cronica                     J320',
    'J321': 'Sinusite frontal cronica                     J321',
    'J322': 'Sinusite etmoidal cronica                    J322',
    'J323': 'Sinusite esfenoidal cronica                  J323',
    'J324': 'Pansinusite cronica                          J324',
    'J328': 'Outr sinusites cronicas                      J328',
    'J329': 'Sinusite cronica NE                          J329',
    'J33': 'Polipo nasal                                 J33 -J339',
    'J330': 'Polipo da cavidade nasal                     J330',
    'J331': 'Degeneracao polipoide do seio paranasal      J331',
    'J338': 'Outr polipos do seio paranasal               J338',
    'J339': 'Polipo nasal NE                              J339',
    'J34': 'Outr transt do nariz e dos seios paranasais  J34 -J349',
    'J340': 'Abscesso furunculo e antraz do nariz         J340',
    'J341': 'Cisto e mucocele do nariz e seio paranasal   J341',
    'J342': 'Desvio do septo nasal                        J342',
    'J343': 'Hipertrofia dos cornetos nasais              J343',
    'J348': 'Outr transt espec nariz e seios paranasais   J348',
    'J35': 'Doenc cronicas das amigdalas e das adenoides J35 -J359',
    'J350': 'Amigdalite cronica                           J350',
    'J351': 'Hipertrofia das amigdalas                    J351',
    'J352': 'Hipertrofia das adenoides                    J352',
    'J353': 'Hipertrofia amigdalas c/hipertrof adenoides  J353',
    'J358': 'Outr doenc cronicas amigdalas e adenoides    J358',
    'J359': 'Doenc das amigdalas e das adenoides NE       J359',
    'J36': 'Abscesso periamigdaliano                     J36 -J369',
    'J37': 'Laringite e laringotraqueite cronicas        J37 -J379',
    'J370': 'Laringite cronica                            J370',
    'J371': 'Laringotraqueite cronica                     J371',
    'J38': 'Doenc das cordas vocais e da laringe NCOP    J38 -J389',
    'J380': 'Paralisia das cordas vocais e da laringe     J380',
    'J381': 'Polipo das cordas vocais e da laringe        J381',
    'J382': 'Nodulos das cordas vocais                    J382',
    'J383': 'Outr doenc das cordas vocais                 J383',
    'J384': 'Edema da laringe                             J384',
    'J385': 'Espasmo da laringe                           J385',
    'J386': 'Estenose da laringe                          J386',
    'J387': 'Outr doenc da laringe                        J387',
    'J39': 'Outr doenc das vias aereas super             J39 -J399',
    'J390': 'Abscesso retrofaringeo e parafaringeo        J390',
    'J391': 'Outr abscessos da faringe                    J391',
    'J392': 'Outr doenc da faringe                        J392',
    'J393': 'Reacao hipersensib vias aereas super loc NE  J393',
    'J398': 'Outr doenc espec das vias aereas super       J398',
    'J399': 'Doenc NE das vias aereas super               J399',
    'J40': 'Bronquite NE como aguda ou cronica           J40 -J409',
    'J41': 'Bronquite cronica simples e a mucopurulenta  J41 -J419',
    'J410': 'Bronquite cronica simples                    J410',
    'J411': 'Bronquite cronica mucopurulenta              J411',
    'J418': 'Bronquite cron mista simples e mucopurulenta J418',
    'J42': 'Bronquite cronica NE                         J42 -J429',
    'J43': 'Enfisema                                     J43 -J439',
    'J430': 'Sindr de MacLeod                             J430',
    'J431': 'Enfisema panlobular                          J431',
    'J432': 'Enfisema centrolobular                       J432',
    'J438': 'Outr form de enfisema                        J438',
    'J439': 'Enfisema NE                                  J439',
    'J44': 'Outr doenc pulmonares obstrutivas cronicas   J44 -J449',
    'J440': 'Doen pulm obs cron c/inf resp ag tr resp inf J440',
    'J441': 'Doenc pulmonar obstr cron c/exacerb aguda NE J441',
    'J448': 'Outr form espec doenc pulmonar obstrut cron  J448',
    'J449': 'Doenc pulmonar obstrutiva cronica NE         J449',
    'J45': 'Asma                                         J45 -J459',
    'J450': 'Asma predom alergica                         J450',
    'J451': 'Asma nao-alergica                            J451',
    'J458': 'Asma mista                                   J458',
    'J459': 'Asma NE                                      J459',
    'J46': 'Estado de mal asmatico                       J46 -J469',
    'J47': 'Bronquectasia                                J47 -J479',
    'J60': 'Pneumoconiose dos mineiros de carvao         J60 -J609',
    'J61': 'Pneumoconiose dev amianto outr fibr minerais J61 -J619',
    'J62': 'Pneumoconiose dev poeira que cont silica     J62 -J629',
    'J620': 'Pneumoconiose dev po de talco                J620',
    'J628': 'Pneumoconiose dev outr poeiras q cont silica J628',
    'J63': 'Pneumoconiose dev outr poeiras inorganicas   J63 -J639',
    'J630': 'Aluminose                                    J630',
    'J631': 'Fibrose causada p/bauxita                    J631',
    'J632': 'Beriliose                                    J632',
    'J633': 'Fibrose causada p/grafite                    J633',
    'J634': 'Siderose                                     J634',
    'J635': 'Estanose                                     J635',
    'J638': 'Pneumoconiose dev outr poeira inorg espec    J638',
    'J64': 'Pneumoconiose NE                             J64 -J649',
    'J65': 'Pneumoconiose assoc c/tuberc                 J65 -J659',
    'J66': 'Doenc das vias aereas dev poeiras org espec  J66 -J669',
    'J660': 'Bissinose                                    J660',
    'J661': 'Doenc dos cardadores de linho                J661',
    'J662': 'Canabinose                                   J662',
    'J668': 'Doenc vias aereas dev outr poeiras org espec J668',
    'J67': 'Pneumonite hipersensibilid dev poeiras org   J67 -J679',
    'J670': 'Pulmao de fazendeiro                         J670',
    'J671': 'Bagacose                                     J671',
    'J672': 'Pulmao dos criadores de passaros             J672',
    'J673': 'Suberose                                     J673',
    'J674': 'Pulmao dos trabalhadores do malte            J674',
    'J675': 'Pulmao dos que trabalham c/cogumelos         J675',
    'J676': 'Pulmao dos cortadores de casca do bordo      J676',
    'J677': 'Doenc pulmonar dev sist ar cond umidif ar    J677',
    'J678': 'Pneumonites hipersens dev outr poeiras org   J678',
    'J679': 'Pneumonite hipersensibilid dev poeira org NE J679',
    'J68': 'Afecc resp dev inal prod quim gas fumac vap  J68 -J689',
    'J680': 'Bronquit pneumon dev prod quim gas fumac vap J680',
    'J681': 'Edema pulm dev prod quim gas fumac vap       J681',
    'J682': 'Infl via aer sup dev pr qui gas fum vap NCOP J682',
    'J683': 'Out af resp ag subag dev pro qui gas fum vap J683',
    'J684': 'Afecc resp cron dev prod quim gas fum vap    J684',
    'J688': 'Outr afecc resp dev prod quim gas fum vap    J688',
    'J689': 'Afecc resp NE dev prod quim gases fumac vap  J689',
    'J69': 'Pneumonite dev solidos e liquidos            J69 -J699',
    'J690': 'Pneumonite dev alimento ou vomito            J690',
    'J691': 'Pneumonite dev oleos e essencias             J691',
    'J698': 'Pneumonite dev outr solidos e liquidos       J698',
    'J70': 'Afeccoes respirat dev outr agentes externos  J70 -J709',
    'J700': 'Manifestacoes pulmonares agudas dev radiacao J700',
    'J701': 'Manif pulmonares cron e outr dev radiacao    J701',
    'J702': 'Transt pulm interstic agudos induz p/droga   J702',
    'J703': 'Transt pulm interstic cronicos induz p/droga J703',
    'J704': 'Transt pulm intersticial NE induz p/droga    J704',
    'J708': 'Afecc respirat dev outr agent externos espec J708',
    'J709': 'Afeccoes respirat dev agentes externos NE    J709',
    'J80': 'Sindr do desconforto respirat do adulto      J80 -J809',
    'J81': 'Edema pulmonar NE de outr form               J81 -J819',
    'J82': 'Eosinofilia pulmonar NCOP                    J82 -J829',
    'J84': 'Outr doenc pulmonares intersticiais          J84 -J849',
    'J840': 'Afeccoes alveolares e parieto-alveolares     J840',
    'J841': 'Outr doenc pulm intersticiais c/fibrose      J841',
    'J848': 'Outr doenc pulmonares intersticiais espec    J848',
    'J849': 'Doenc pulmonar intersticial NE               J849',
    'J85': 'Abscesso do pulmao e do mediastino           J85 -J859',
    'J850': 'Gangrena e necrose do pulmao                 J850',
    'J851': 'Abscesso do pulmao c/pneumonia               J851',
    'J852': 'Abscesso do pulmao s/pneumonia               J852',
    'J853': 'Abscesso do mediastino                       J853',
    'J86': 'Piotorax                                     J86 -J869',
    'J860': 'Piotorax c/fistula                           J860',
    'J869': 'Piotorax s/fistula                           J869',
    'J90': 'Derrame pleural NCOP                         J90 -J909',
    'J91': 'Derrame pleural em afeccoes COP              J91 -J919',
    'J92': 'Placas pleurais                              J92 -J929',
    'J920': 'Placas pleurais c/presenca de amianto        J920',
    'J929': 'Placas pleurais cem presenca de amianto      J929',
    'J93': 'Pneumotorax                                  J93 -J939',
    'J930': 'Pneumotorax de tensao espontaneo             J930',
    'J931': 'Outr form de pneumotorax espontaneo          J931',
    'J938': 'Outr tipos de pneumotorax espec              J938',
    'J939': 'Pneumotorax NE                               J939',
    'J94': 'Outr afeccoes pleurais                       J94 -J949',
    'J940': 'Derrame quiloso ou quiliforme                J940',
    'J941': 'Fibrotorax                                   J941',
    'J942': 'Hemotorax                                    J942',
    'J948': 'Outr afeccoes pleurais espec                 J948',
    'J949': 'Afeccao pleural NE                           J949',
    'J95': 'Afeccoes respirat pos-proced NCOP            J95 -J959',
    'J950': 'Mau funcionamento de traqueostomia           J950',
    'J951': 'Insuf pulmonar aguda subseq cirurg toracica  J951',
    'J952': 'Insuf pulm aguda subseq cirurg nao toracica  J952',
    'J953': 'Insuf pulmonar cronica pos-cirurgica         J953',
    'J954': 'Sindr de Mendelson                           J954',
    'J955': 'Estenose subglotica pos-proced               J955',
    'J958': 'Outr transt respirat pos-proced              J958',
    'J959': 'Transt respirat pos-proced NE                J959',
    'J96': 'Insuf respirat NCOP                          J96 -J969',
    'J960': 'Insuf respirat aguda                         J960',
    'J961': 'Insuf respirat cronica                       J961',
    'J969': 'Insuf respirat NE                            J969',
    'J98': 'Outr transt respirat                         J98 -J989',
    'J980': 'Outr doenc dos bronquios NCOP                J980',
    'J981': 'Colapso pulmonar                             J981',
    'J982': 'Enfisema intersticial                        J982',
    'J983': 'Enfisema compensatorio                       J983',
    'J984': 'Outr transt pulmonares                       J984',
    'J985': 'Doenc do mediastino NCOP                     J985',
    'J986': 'Transt do diafragma                          J986',
    'J988': 'Outr transt respirat espec                   J988',
    'J989': 'Transt respirat NE                           J989',
    'J99': 'Transt respirat em doenc COP                 J99 -J999',
    'J990': 'Doenc pulmonar reumatoide                    J990',
    'J991': 'Transt respirat outr doenc sist tec conj COP J991',
    'J998': 'Transt respirat em outr doenc COP            J998',
}
diseases_cols = [
    'DIAG_PRINC',
    'DIAG_SECUN',
    'DIAGSEC1',
    'DIAGSEC2',
    'DIAGSEC3',
    'DIAGSEC4',
    'DIAGSEC5',
    'DIAGSEC6',
    'DIAGSEC7',
    'DIAGSEC8',
    'DIAGSEC9'
]

### 6.4 List dbc files

In [31]:
path = f"{PATHS['IN']}/{FOLDERS['DSUS']}/RD/**/*.dbc"
f_dsus = glob.glob(path, recursive=True)
show_enum(f_dsus)

{0: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0801.dbc',
 1: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0802.dbc',
 2: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0803.dbc',
 3: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0804.dbc',
 4: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0805.dbc',
 5: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0806.dbc',
 6: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0807.dbc',
 7: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0808.dbc',
 8: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0809.dbc',
 9: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0810.dbc',
 10: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0811.dbc',
 11: '../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0812.dbc',
 12: '../datas/raw/mr-ssa/datasus/RD/BA/09/RDBA0901.dbc',
 13: '../datas/raw/mr-ssa/datasus/RD/BA/09/RDBA0902.dbc',
 14: '../datas/raw/mr-ssa/datasus/RD/BA/09/RDBA0903.dbc',
 15: '../datas/raw/mr-ssa/datasus/RD/BA/09/RDBA0904.dbc',
 16: '../datas/raw/mr-ssa/datasus/RD/BA/09/RDBA0905.dbc',
 17: '../datas/raw/mr-ss

### 6.5 [DATASUS] Generating csv

In [35]:
df = pd.DataFrame()

for file in f_dsus:
    print(f'\nStarting {file}')
    
    # Open dataframe
    df_dsus = read_dbc(file, encoding='iso-8859-1')

    print('[INFO] Removes unused columns')
    drop_cols(df_dsus, unused_cols)

    print('[INFO] Get only metropolitan region of Salvador')
    df_dsus = _filter(df_dsus, cities_cols, cities_mrs)

    print('[INFO] Get only respiratory disease')
    df_dsus = _filter(df_dsus, diseases_cols, resp_diseases)
    
    print('[INFO] Format dates')
    df_dsus['NASC'] = pd.to_datetime(df_dsus['NASC'])
    df_dsus['DT_INTER'] = pd.to_datetime(df_dsus['DT_INTER'])

    df = pd.concat([df, df_dsus], ignore_index=True)

    print(f'Merging {file}\n')
    
# Generate output name
f_out = f"{PATHS['OUT']}/{FOLDERS['DSUS']}/RD.csv.gz"

# Generate gzip output file
df.to_csv(f_out, compression='gzip') #index=False

print('Ending all.')


Starting ../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0801.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0801.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0802.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0802.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0803.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0803.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/08/RDBA0804.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/08/RD

[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/10/RDBA1010.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/10/RDBA1011.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/10/RDBA1011.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/10/RDBA1012.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/10/RDBA1012.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/11/RDBA1101.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/11/RDBA1101.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/11/R

Merging ../datas/raw/mr-ssa/datasus/RD/BA/13/RDBA1306.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/13/RDBA1307.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/13/RDBA1307.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/13/RDBA1308.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/13/RDBA1308.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/13/RDBA1309.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/13/RDBA1309.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/13/RDBA1310.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] F

[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/16/RDBA1603.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/16/RDBA1604.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/16/RDBA1604.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/16/RDBA1605.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/16/RDBA1605.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/16/RDBA1606.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/16/RDBA1606.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/16/R

Merging ../datas/raw/mr-ssa/datasus/RD/BA/18/RDBA1811.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/18/RDBA1812.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/18/RDBA1812.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/19/RDBA1901.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/19/RDBA1901.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/19/RDBA1902.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] Format dates
Merging ../datas/raw/mr-ssa/datasus/RD/BA/19/RDBA1902.dbc


Starting ../datas/raw/mr-ssa/datasus/RD/BA/19/RDBA1903.dbc
[INFO] Removes unused columns
[INFO] Get only metropolitan region of Salvador
[INFO] Get only respiratory disease
[INFO] F